In [1]:
import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import growthepieapi_utils as gtp
import l2beat_utils as ltwo
sys.path.pop()

import numpy as np
import pandas as pd

In [2]:
# # # Usage
gtp_api = gtp.get_growthepie_api_data()
# # print(merged_df.sample(5))
gtp_meta_api = gtp.get_growthepie_api_meta()

In [3]:
l2beat_df = ltwo.get_all_l2beat_data()
l2beat_meta = ltwo.get_l2beat_metadata()
l2beat_meta['chain'] = l2beat_meta['slug']
print(l2beat_meta.sample(5))

/Users/michaelsilberling/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/l2beat_utils.py:147: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)


     layer      name chainId explorerUrl           category      slug  \
85  layer2     Swell    None        None           Validium     swell   
51  layer2  Loopring    None        None          ZK Rollup  loopring   
70  layer2   Paradex    None        None          ZK Rollup   paradex   
21  layer2      Coti    None        None  Optimistic Rollup      coti   
88  layer2       Ten    None        None  Optimistic Rollup       ten   

    provider is_upcoming     chain  
85   Polygon        True     swell  
51  Loopring       False  loopring  
70  Starknet       False   paradex  
21      None        True      coti  
88      None        True       ten  


/Users/michaelsilberling/Documents/GitHub/op-analytics/other_chains_tracking/../helper_functions/l2beat_utils.py:147: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = pd.concat([df, pd.DataFrame(data_list)], ignore_index=True)


In [4]:
# display(l2beat_meta[l2beat_meta['layer']=='layer2'].sample(10))

In [5]:
# display(l2beat_meta.sample(5))

In [6]:
combined_l2b_df = l2beat_df.merge(l2beat_meta[['chain','layer','chainId','provider','category','is_upcoming']], on='chain',how='left')
# combined_l2b_df.tail(5)

In [7]:
combined_gtp_df = gtp_api.merge(gtp_meta_api[['origin_key','chain_name']], on='origin_key',how='left')
# combined_gtp_df.sample(5)

In [8]:
# Check Columns
# Assuming combined_gtp_df is your DataFrame
column_names = combined_gtp_df.columns

for col in column_names:
    if col.endswith('_usd'):
        # Construct the new column name by replacing '_usd' with '_eth'
        new_col_name = col.replace('_usd', '_eth')
        
        # Check if the new column name exists in the DataFrame
        if new_col_name not in combined_gtp_df.columns:
            # If it doesn't exist, create the column and fill it with nan values
            combined_gtp_df[new_col_name] = np.nan

In [9]:
# print(combined_gtp_df.dtypes)
# print(l2beat_df.dtypes)
# combined_gtp_df.sample(5)

In [10]:
# export
folder = 'outputs/'
combined_gtp_df.to_csv(folder + 'growthepie_l2_activity.csv', index = False)
gtp_meta_api.to_csv(folder + 'growthepie_l2_metadata.csv', index = False)
combined_l2b_df.to_csv(folder + 'l2beat_l2_activity.csv', index = False)
l2beat_meta.to_csv(folder + 'l2beat_l2_metadata.csv', index = False)
# Post to Dune API
d.write_dune_api_from_pandas(combined_gtp_df, 'growthepie_l2_activity',\
                             'L2 Usage Activity from GrowThePie')
d.write_dune_api_from_pandas(gtp_meta_api, 'growthepie_l2_metadata',\
                             'L2 Metadata from GrowThePie')
d.write_dune_api_from_pandas(combined_l2b_df, 'l2beat_l2_activity',\
                             'L2 Usage Activity from L2Beat')
d.write_dune_api_from_pandas(l2beat_meta, 'l2beat_l2_metadata',\
                             'L2 Metadata from L2Beat')


table at: dune.oplabspbc.dataset_growthepie_l2_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"growthepie_l2_activity"}'
table at: dune.oplabspbc.dataset_growthepie_l2_metadata
Response status code: 200
Response content: b'{"success":true,"table_name":"growthepie_l2_metadata"}'
table at: dune.oplabspbc.dataset_l2beat_l2_activity
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_activity"}'
table at: dune.oplabspbc.dataset_l2beat_l2_metadata
Response status code: 200
Response content: b'{"success":true,"table_name":"l2beat_l2_metadata"}'
